<a href="https://colab.research.google.com/github/Naveenand/Computer-vision/blob/main/langchain_tools_custum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai  langchain-experimental yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.6/229.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-

In [2]:
import yfinance as yf
from datetime import datetime, timedelta


/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [3]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"New_startup project - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [4]:
os.environ["LANGCHAIN_API_KEY"] = "your_api_key"

In [5]:
def get_current_stock_price(ticker):
  """Method to get current stock price"""
  ticker_data = yf.Ticker(ticker)
  recent = ticker_data.history(period= '1d')
  return {'price': recent.iloc[0]['Close'],'currency': ticker_data.info['currency']}

In [6]:
def get_stock_performance(ticker,days):
  """Method to get stock price change in percentage"""

  past_date = datetime.today() - timedelta(days = days)
  ticker_data = yf.Ticker(ticker)
  history = ticker_data.history(start = past_date)
  old_price = history.iloc[0]['Close']
  current_price = history.iloc[-1]['Close']
  return {'Percent_change': ((current_price - old_price)/ old_price) * 100}

In [7]:
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

In [8]:
from typing import List

In [9]:
class CurrentStockPriceInput(BaseModel):
  """Inputs for get_current_stock_price"""

  ticker: str = Field(description = "Ticker symbol of the stock")

class CurrentStockPriceTool(BaseTool):
  name = "get_current_stock_price"
  description = """
      useful when you want to get current stock price.
      you should enter the stock ticker symbol recongnized by the yahoo finance
      output will be stock current price and stock price currency
      """

  args_schema: Type[BaseModel] = CurrentStockPriceInput

  def _run(self,ticker : str):
    price_response = get_current_stock_price(ticker)
    return price_response


  def _arun(self,ticker:str):
    raise NotImplementedError('get_current_stock_price does not support async')

In [10]:
class StockPercentChangeInput(BaseModel):
  """Inputs for get_stock_performance"""

  ticker: str = Field(description = "Ticker symbol of the stock")
  days : int = Field(description = "Timedelta days to get past date from current date")



class StockPercentChangeTool(BaseTool):

  name = 'get_stock_performance'
  description = """
      useful when you want to check performance of the stock.
      you should enter the stock ticker symbol recongnized by the yahoo finance.
      you should enter days as number of days from today from which performance needs to be check
      output will be the change in the stock price represented as a percentage.
      """
  args_schema: Type[BaseModel] = StockPercentChangeInput

  def _run(self, ticker:str, days : int):
    response = get_stock_performance(ticker,days)
    return response

  def _arun(self,ticker:str):
    raise NotImplementedError("get_stock_performance does not support async")

In [11]:
class VolatileScoreInput(BaseModel):
    """Inputs for calculate_volatile_score"""

    stock_ticker: str = Field(description="Ticker symbol of the stock")
    days: int = Field(description="Number of days to calculate volatility score")

class VolatileScoreTool(BaseTool):
    name = 'calculate_volatile_score'
    description = """
        Useful when you want to calculate the volatility score of a stock.
        Enter the stock ticker symbol recognized by Yahoo Finance and the number of days.
        The output will be the volatility score represented as a percentage.
        """
    args_schema: Type[BaseModel] = VolatileScoreInput

    def _run(self, stock_ticker: str, days: int):
        stock_data = yf.download(stock_ticker, period=f"{days}d")
        stock_data['Daily_Return'] = stock_data['Close'].pct_change()
        volatility_score = stock_data['Daily_Return'].std()
        return {'volatility_score': volatility_score * 100}

    def _arun(self, stock_ticker: str):
        raise NotImplementedError("calculate_volatile_score does not support async")


In [12]:
class MultiStockPriceInput(BaseModel):
    """Inputs for get_current_stock_price for multiple stocks"""

    tickers: List[str] = Field(description="List of stock ticker symbols")

class MultiStockPriceTool(BaseTool):
    name = "get_multi_stock_prices"
    description = """
        Useful when you want to get current stock prices for multiple stocks.
        Enter a list of stock ticker symbols recognized by Yahoo Finance.
        The output will be a list of dictionaries, each containing the current price and currency.
        """
    args_schema: Type[BaseModel] = MultiStockPriceInput

    def _run(self, tickers: List[str]):
        prices = []
        for ticker in tickers:
            price_response = get_current_stock_price(ticker)
            prices.append({'ticker': ticker, 'price': price_response['price'], 'currency': price_response['currency']})
        return prices

    def _arun(self, tickers: List[str]):
        raise NotImplementedError('get_multi_stock_prices does not support async')


In [13]:
class CalculateInvestmentReturnInput(BaseModel):
    """Inputs for calculate_investment_return"""

    tickers: List[str] = Field(description="List of stock ticker symbols")
    days: int = Field(description="Number of days to calculate investment return")
    initial_investment: float = Field(description="Initial investment amount")

class CalculateInvestmentReturnTool(BaseTool):
    name = 'calculate_investment_return'
    description = """
        Useful when you want to calculate the investment return for multiple stocks.
        Enter a list of stock ticker symbols recognized by Yahoo Finance.
        Enter the number of days and the initial investment amount.
        The output will be a list of dictionaries, each containing the total return on investment.
        """
    args_schema: Type[BaseModel] = CalculateInvestmentReturnInput

    def _run(self, tickers: List[str], days: int, initial_investment: float):
        returns = []
        for ticker in tickers:
            ticker_data = yf.Ticker(ticker)
            history = ticker_data.history(period=f'{days}d')
            old_price = history.iloc[0]['Close']
            current_price = history.iloc[-1]['Close']
            percent_change = ((current_price - old_price) / old_price) * 100
            total_return = initial_investment * (1 + percent_change / 100)
            returns.append({'ticker': ticker, 'total_return': total_return})
        return returns

    def _arun(self, tickers: List[str]):
        raise NotImplementedError("calculate_investment_return does not support async")


In [14]:
from typing import Type, List
from pydantic import BaseModel, Field
import yfinance as yf

class GovernanceMetricsInput(BaseModel):
    """Inputs for get_governance_metrics for multiple companies"""

    symbols: List[str] = Field(description="List of stock symbols")

class GovernanceMetricsTool(BaseTool):
    name = 'get_governance_metrics'
    description = """
        Useful when you want to get governance metrics for multiple companies.
        Enter a list of stock symbols recognized by Yahoo Finance.
        The output will be a list of dictionaries, each containing governance metrics.
        """
    args_schema: Type[BaseModel] = GovernanceMetricsInput

    def _run(self, symbols: List[str]):
        governance_metrics_list = []
        for symbol in symbols:
            governance_metrics = self._get_governance_metrics(symbol)
            governance_metrics_list.append({'symbol': symbol, 'governance_metrics': governance_metrics})
        return governance_metrics_list

    def _get_governance_metrics(self, symbol):
        try:
            # Download stock information from Yahoo Finance
            stock_data = yf.Ticker(symbol)

            # Company Information
            company_info = stock_data.info

            # Extract Governance Metrics
            governance_metrics = {
                'Audit Risk': company_info.get('auditRisk', ''),
                'Board Risk': company_info.get('boardRisk', ''),
                'Compensation Risk': company_info.get('compensationRisk', ''),
                'Shareholder Rights Risk': company_info.get('shareHolderRightsRisk', ''),
                'Overall Risk': company_info.get('overallRisk', '')
            }

            return governance_metrics

        except Exception as e:
            return {'Do Not have information': str(e)}

    def _arun(self, symbols: List[str]):
        raise NotImplementedError('get_governance_metrics does not support async')

In [15]:
class FinancialRatiosInput(BaseModel):
    """Inputs for get_financial_ratios for multiple companies"""

    stock_symbols: List[str] = Field(description="List of stock symbols")

class FinancialRatiosTool(BaseTool):
    name = 'get_financial_ratios'
    description = """
        Useful when you want to get financial ratios for multiple companies.
        Enter a list of stock symbols recognized by Yahoo Finance.
        The output will be a list of dictionaries, each containing financial ratios.
        """
    args_schema: Type[BaseModel] = FinancialRatiosInput

    def _run(self, stock_symbols: List[str]):
        financial_ratios_list = []
        for stock_symbol in stock_symbols:
            try:
                financial_ratios = self._get_financial_ratios(stock_symbol)
                financial_ratios_list.append({'stock_symbol': stock_symbol, 'financial_ratios': financial_ratios})
            except Exception as e:
                financial_ratios_list.append({'stock_symbol': stock_symbol, 'error': str(e)})

        return financial_ratios_list

    def _get_financial_ratios(self, stock_symbol):
        stock = yf.Ticker(stock_symbol)

        financial_ratios = {
            'Earnings Per Share (EPS)': stock.info.get('trailingEps'),
            'EPS Growth (%)': stock.info.get('earningsGrowth'),
            'Price-to-Earnings (P/E) Ratio (%)': stock.info.get('trailingPE'),
            'Price/Earnings-to-Growth (PEG) Ratio (%)': stock.info.get('pegRatio'),
            'Operating Margin (%)': stock.info.get('operatingMargins'),
            'Return on Assets (ROA) (%)': stock.info.get('returnOnAssets'),
            'Return on Equity (ROE) (%)': stock.info.get('returnOnEquity'),
            'Current Ratio': stock.info.get('currentRatio'),
            'Quick Ratio': stock.info.get('quickRatio'),
            'Dividend Per Share ($)': stock.info.get('dividendRate'),
            'Dividend Growth (%)': stock.info.get('dividendYield'),
            'Dividend Cover (%)': stock.info.get('payoutRatio'),
        }

        return financial_ratios

    def _arun(self, stock_symbols: List[str]):
        raise NotImplementedError('get_financial_ratios does not support async')


In [16]:
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

In [17]:
CurrentStockPriceTool()

CurrentStockPriceTool()

In [18]:
StockPercentChangeTool()

StockPercentChangeTool()

In [19]:
VolatileScoreTool()

VolatileScoreTool()

In [20]:
MultiStockPriceTool()

MultiStockPriceTool()

In [21]:
llm = ChatOpenAI(openai_api_key = 'your_api_key')


tools = [MultiStockPriceTool(), StockPercentChangeTool(),VolatileScoreTool(),CalculateInvestmentReturnTool(),GovernanceMetricsTool()]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [23]:
agent = initialize_agent(tools, llm, agent = AgentType.OPENAI_FUNCTIONS, verbose = True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [24]:
agent.run("waht is the current price of apple,microsoft,google and nvidia stock ? how much money will return If I invest 1000 in each company one year before? which company gives highest return?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `get_multi_stock_prices` with `{'tickers': ['AAPL', 'MSFT', 'GOOGL', 'NVDA']}`


[{'ticker': 'AAPL', 'price': 194.5, 'currency': 'USD'}, {'ticker': 'MSFT', 'price': 402.55999755859375, 'currency': 'USD'}, {'ticker': 'GOOGL', 'price': 148.6999969482422, 'currency': 'USD'}, {'ticker': 'NVDA', 'price': 613.6199951171875, 'currency': 'USD'}]
Invoking: `calculate_investment_return` with `{'tickers': ['AAPL', 'MSFT', 'GOOGL', 'NVDA'], 'days': 365, 'initial_investment': 1000}`


[{'ticker': 'AAPL', 'total_return': 1162.7491950357023}, {'ticker': 'MSFT', 'total_return': 1421.9863243989855}, {'ticker': 'GOOGL', 'total_return': 1251.2622141577017}, {'ticker': 'NVDA', 'total_return': 3423.4302644917457}]The current prices of the stocks are as follows:

- Apple (AAPL): $194.50 USD
- Microsoft (MSFT): $402.56 USD
- Google (GOOGL): $148.70 USD
- NVIDIA (NVDA): $613.62 USD

If you had invested $1000 in each company one year ago, the total returns wou

'The current prices of the stocks are as follows:\n\n- Apple (AAPL): $194.50 USD\n- Microsoft (MSFT): $402.56 USD\n- Google (GOOGL): $148.70 USD\n- NVIDIA (NVDA): $613.62 USD\n\nIf you had invested $1000 in each company one year ago, the total returns would be:\n\n- Apple (AAPL): $1162.75 USD\n- Microsoft (MSFT): $1421.99 USD\n- Google (GOOGL): $1251.26 USD\n- NVIDIA (NVDA): $3423.43 USD\n\nThe company with the highest return on investment is NVIDIA (NVDA), with a total return of $3423.43 USD.'

In [ ]:
agent.run("what the current price of apple,microsoft and nvidia ? and Compensation Risk?")



> Entering new AgentExecutor chain...

Invoking: `get_multi_stock_prices` with `{'tickers': ['AAPL', 'MSFT', 'NVDA']}`


[{'ticker': 'AAPL', 'price': 193.88999938964844, 'currency': 'USD'}, {'ticker': 'MSFT', 'price': 396.510009765625, 'currency': 'USD'}, {'ticker': 'NVDA', 'price': 596.5399780273438, 'currency': 'USD'}]
Invoking: `get_governance_metrics` with `{'symbols': ['AAPL', 'MSFT', 'NVDA']}`


[{'symbol': 'AAPL', 'governance_metrics': {'Audit Risk': 4, 'Board Risk': 1, 'Compensation Risk': 6, 'Shareholder Rights Risk': 1, 'Overall Risk': 1}}, {'symbol': 'MSFT', 'governance_metrics': {'Audit Risk': 6, 'Board Risk': 4, 'Compensation Risk': 2, 'Shareholder Rights Risk': 2, 'Overall Risk': 2}}, {'symbol': 'NVDA', 'governance_metrics': {'Audit Risk': 7, 'Board Risk': 10, 'Compensation Risk': 1, 'Shareholder Rights Risk': 8, 'Overall Risk': 8}}]The current prices of Apple, Microsoft, and Nvidia are as follows:
- Apple (AAPL): $193.89 USD
- Microsoft (MSFT): $396.51 USD
- Nvidia (NV

'The current prices of Apple, Microsoft, and Nvidia are as follows:\n- Apple (AAPL): $193.89 USD\n- Microsoft (MSFT): $396.51 USD\n- Nvidia (NVDA): $596.54 USD\n\nRegarding compensation risk, here are the ratings for each company:\n- Apple (AAPL): 6 out of 10\n- Microsoft (MSFT): 2 out of 10\n- Nvidia (NVDA): 1 out of 10\n\nA higher compensation risk rating indicates a higher level of risk associated with executive compensation practices.'

In [ ]:
agent.run("how apple,microsoft and nvidia performed over past one year? and how much return I get if I invested 1000 usd in each three companies ?")



> Entering new AgentExecutor chain...

Invoking: `get_stock_performance` with `{'ticker': 'AAPL', 'days': 365}`


{'Percent_change': 36.511017429352385}
Invoking: `get_stock_performance` with `{'ticker': 'MSFT', 'days': 365}`


{'Percent_change': 65.80302903960262}
Invoking: `get_stock_performance` with `{'ticker': 'NVDA', 'days': 365}`


{'Percent_change': 210.09948025673947}Over the past one year, Apple (AAPL) has performed with a 36.51% increase, Microsoft (MSFT) has performed with a 65.80% increase, and NVIDIA (NVDA) has performed with a 210.10% increase.

If you had invested $1000 in each of these three companies, your total return would be as follows:
- Apple: $365.11
- Microsoft: $658.03
- NVIDIA: $2100.99

> Finished chain.


'Over the past one year, Apple (AAPL) has performed with a 36.51% increase, Microsoft (MSFT) has performed with a 65.80% increase, and NVIDIA (NVDA) has performed with a 210.10% increase.\n\nIf you had invested $1000 in each of these three companies, your total return would be as follows:\n- Apple: $365.11\n- Microsoft: $658.03\n- NVIDIA: $2100.99'

In [ ]:
agent.run("explain about finance bonds investment ?")



> Entering new AgentExecutor chain...
Bonds are a type of investment where an investor lends money to a government or a corporation for a fixed period of time, typically ranging from a few months to several years. In return, the issuer of the bond promises to pay the investor a specified amount of interest at regular intervals, and to repay the principal amount (the initial investment) when the bond matures.

Bonds are considered to be a relatively safer investment compared to stocks because they offer a fixed income stream and are generally less volatile. They are often used by investors who are looking for a more stable and predictable return on their investment.

Here are some key points to understand about bond investments:

1. Types of Bonds: There are various types of bonds available in the market, including government bonds, corporate bonds, municipal bonds, and international bonds. Each type of bond carries different levels of risk and return.

2. Coupon Rate: The coupon rate

"Bonds are a type of investment where an investor lends money to a government or a corporation for a fixed period of time, typically ranging from a few months to several years. In return, the issuer of the bond promises to pay the investor a specified amount of interest at regular intervals, and to repay the principal amount (the initial investment) when the bond matures.\n\nBonds are considered to be a relatively safer investment compared to stocks because they offer a fixed income stream and are generally less volatile. They are often used by investors who are looking for a more stable and predictable return on their investment.\n\nHere are some key points to understand about bond investments:\n\n1. Types of Bonds: There are various types of bonds available in the market, including government bonds, corporate bonds, municipal bonds, and international bonds. Each type of bond carries different levels of risk and return.\n\n2. Coupon Rate: The coupon rate is the interest rate that the 

In [ ]:
import yfinance as yf

def get_financial_ratios(stock_ticker):
    # Download financial data for the given stock ticker
    stock = yf.Ticker(stock_ticker)

    # Get key financial data
    financial_data = stock.info

    # Calculate financial ratios
    pe_ratio = financial_data['trailingPE'] if 'trailingPE' in financial_data else None
    pb_ratio = financial_data['priceToBook'] if 'priceToBook' in financial_data else None
    debt_to_equity_ratio = financial_data['debtToEquity'] if 'debtToEquity' in financial_data else None
    free_cash_flow = financial_data['freeCashflow'] if 'freeCashflow' in financial_data else None
    peg_ratio = financial_data['pegRatio'] if 'pegRatio' in financial_data else None

    return {
        'P/E Ratio': pe_ratio,
        'P/B Ratio': pb_ratio,
        'Debt-to-Equity Ratio': debt_to_equity_ratio,
        'Free Cash Flow': free_cash_flow,
        'PEG Ratio': peg_ratio
    }

# Example usage:
ticker = "AAPL"  # Replace with the desired stock ticker
ratios = get_financial_ratios(ticker)
print("Financial Ratios for", ticker)
for ratio_name, ratio_value in ratios.items():
    print(f"{ratio_name}: {ratio_value}")


Financial Ratios for AAPL
P/E Ratio: 31.62969
P/B Ratio: 48.50888
Debt-to-Equity Ratio: 199.418
Free Cash Flow: 82179997696
PEG Ratio: 4.73


In [ ]:
# Download financial data for the given stock ticker
stock = yf.Ticker(ticker)

# Get key financial data
financial_data = stock.info

In [ ]:
financial_data

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '408 996 1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and pod